Import dependencies and configure settings.

In [1]:
import os
import csv
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scipy.stats as stats
import statsmodels.api as sm

import hawkes
import markDistributions as md
import scipy.optimize as op

from pylab import rcParams
from __future__ import division

%matplotlib inline
np.set_printoptions(precision=10)

Load data.

In [2]:
dat = 'AMZN_2012-06-21_34200000_57600000_message_10'
temp = ([],[],[],[],[],[])
with open('../../data/'+dat+'.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        temp[0].append(float(row[0]))
        temp[1].append(int(row[1]))
        temp[2].append(int(row[2]))
        temp[3].append(int(row[3]))
        temp[4].append(int(row[4]))
        temp[5].append(int(row[5]))

orders = {'Time':None, 'Type':None, 'Order ID':None,
        'Volume':None, 'Price':None, 'Direction':None}
orders['Time'] = np.asarray(temp[0])
orders['Type'] = np.asarray(temp[1])
orders['Order ID'] = np.asarray(temp[2])
orders['Volume'] = np.asarray(temp[3])
orders['Price'] = np.asarray(temp[4])
orders['Direction'] = np.asarray(temp[5])
del temp

In [3]:
ttemp = len(orders['Type'])
print("{0} order messages total:".format(len(orders['Type'])))
temp = len(orders['Type'][orders['Type']==1])
print("- {0} LO submissions ({1:.2f}%)".format(temp,100*temp/ttemp))
temp = len(orders['Type'][orders['Type']==2])
print("- {0} partial LO cancellations ({1:.2f}%)".format(temp,100*temp/ttemp))
temp = len(orders['Type'][orders['Type']==3])
print("- {0} LO cancellations ({1:.2f}%)".format(temp,100*temp/ttemp))
temp = len(orders['Type'][orders['Type']==4])
print("- {0} visible LO execution ({1:.2f}%)".format(temp,100*temp/ttemp))
temp = len(orders['Type'][orders['Type']==5])
print("- {0} hidden LO execution ({1:.2f}%)".format(temp,100*temp/ttemp))
temp = len(orders['Type'][orders['Type']==7])
print("- {0} trading halts ({1:.2f}%)".format(temp,100*temp/ttemp))
del ttemp
del temp

269748 order messages total:
- 131954 LO submissions (48.92%)
- 2917 partial LO cancellations (1.08%)
- 123458 LO cancellations (45.77%)
- 8974 visible LO execution (3.33%)
- 2445 hidden LO execution (0.91%)
- 0 trading halts (0.00%)


Extract data into usable form.

In [ ]:
LTC_mask = (orders['Type']==1) | (orders['Type']==3) | (orders['Type']==4)
t = orders['Time'][LTC_mask]
j = orders['Type'][LTC_mask]
j[j==3] = 2
j[j==4] = 3
x = orders['Volume'][LTC_mask]

Initialise a 6-D Hawkes point process and fit the data to the model.

In [ ]:
d = 6
h = hawkes.Hawkes(d=d)

result = h.MLE(t,j,x,method='L-BFGS-B')

The fitted parameters and stability check:

In [ ]:
arr = result["x"]

print("""eta:
        {0} - estimated
        {1} - true\n""".format(arr[:2],np.asarray(eta)))
print("""Q:
        {0} - estimated
        {1} - true\n""".format(arr[2:6],np.asarray(Q)))
print("""alpha:
        {0} - estimated
        {1} - true\n""".format(arr[6:8],np.asarray(alpha)))
print("""alph:
        {0} - estimated
        {1} - true\n""".format(arr[8:10],np.asarray(alph)))
print("""beta:
        {0} - estimated
        {1} - true\n""".format(arr[10:12],np.asarray(beta)))

print("""Spr(Q) of estimated model: {0}""".format(h.ibParam.getSpectralRadius()))